# Tutorial: Chemical-Disease Extraction

## Part I: Candidate Extraction

In this example, we'll be writing an application to extract **chemical-disease** relationships from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate chemical-disease (C-D) relation mentions_ as either true or false.  To do this, we first need a set of such candidates.

In this notebook, we'll use `DDLite` utilities to extract these candidates.  In _part II_, we'll start with the gold set of candidates and just focus on the core candidate classification task.

### Parsing from XML format

We'll start by using `DDLite`'s `DocParser` class to read in Pubmed abstracts from [Pubtator]([Pubtator](http://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi)), where they are stored along with gold chemical and disease mention annotations.

We'll use the `XMLDocParser` class, which allows us to use XPath queries to specify the relevant sections of the XML format.  Note that we are simply newline-concatenating text from the title and abstract together for simplicity.

In [1]:
from ddlite_parser import XMLDocParser
xml_parser = XMLDocParser(path='data/CDR_DevelopmentSet.xml',
    doc='.//document', text='.//passage/text/text()',
    id='.//id/text()', keep_tree=True)
docs = list(xml_parser.parse())
docs[0]

Document(id='6794356', file='CDR_DevelopmentSet.xml', text='Tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant.\nA newborn with massive tricuspid regurgitation, atrial flutter, congestive heart failure, and a high serum lithium level is described. This is the first patient to initially manifest tricuspid regurgitation and atrial flutter, and the 11th described patient with cardiac disease among infants exposed to lithium compounds in the first trimester of pregnancy. Sixty-three percent of these infants had tricuspid valve involvement. Lithium carbonate may be a factor in the increasing incidence of congenital heart disease when taken during early pregnancy. It also causes neurologic depression, cyanosis, and cardiac arrhythmia when consumed prior to delivery.', attribs={'root': <Element document at 0x10b7d4e60>})

### Pre-processing the sentences

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.

In [2]:
from ddlite_parser import SentenceParser
parser = SentenceParser()
sents  = parser.parse_docs(docs)
sent   = sents[0]
sent

Sentence(id='6794356-0', words=[u'Tricuspid', u'valve', u'regurgitation', u'and', u'lithium', u'carbonate', u'toxicity', u'in', u'a', u'newborn', u'infant', u'.'], lemmas=[u'tricuspid', u'valve', u'regurgitation', u'and', u'lithium', u'carbonate', u'toxicity', u'in', u'a', u'newborn', u'infant', u'.'], poses=[u'JJ', u'NN', u'NN', u'CC', u'NN', u'NN', u'NN', u'IN', u'DT', u'JJ', u'NN', u'.'], dep_parents=[3, 3, 0, 3, 7, 7, 3, 11, 11, 11, 3, 3], dep_labels=[u'amod', u'compound', u'ROOT', u'cc', u'compound', u'compound', u'conj', u'case', u'det', u'amod', u'nmod', u'punct'], sent_id=0, doc_id='6794356', text=u'Tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant.', char_offsets=[0, 10, 16, 30, 34, 42, 52, 61, 64, 66, 74, 80], doc_name='CDR_DevelopmentSet.xml')

## Testing `CandidateSpace` and `Matcher`

In [39]:
from ddlite_candidates import Ngrams
from ddlite_matchers import DictionaryMatch

In [40]:
cs = Ngrams(n_max=3)

In [43]:
for ng in cs.apply(sent):
    print ng

<Ngram("Tricuspid valve regurgitation", id=6794356-0:0-28, chars=[0,28], words=[0,2])
<Ngram("valve regurgitation and", id=6794356-0:10-32, chars=[10,32], words=[1,3])
<Ngram("regurgitation and lithium", id=6794356-0:16-40, chars=[16,40], words=[2,4])
<Ngram("and lithium carbonate", id=6794356-0:30-50, chars=[30,50], words=[3,5])
<Ngram("lithium carbonate toxicity", id=6794356-0:34-59, chars=[34,59], words=[4,6])
<Ngram("carbonate toxicity in", id=6794356-0:42-62, chars=[42,62], words=[5,7])
<Ngram("toxicity in a", id=6794356-0:52-64, chars=[52,64], words=[6,8])
<Ngram("in a newborn", id=6794356-0:61-72, chars=[61,72], words=[7,9])
<Ngram("a newborn infant", id=6794356-0:64-79, chars=[64,79], words=[8,10])
<Ngram("newborn infant .", id=6794356-0:66-81, chars=[66,81], words=[9,11])
<Ngram("Tricuspid valve", id=6794356-0:0-14, chars=[0,14], words=[0,1])
<Ngram("valve regurgitation", id=6794356-0:10-28, chars=[10,28], words=[1,2])
<Ngram("regurgitation and", id=6794356-0:16-32, chars=[16,

In [46]:
matcher = DictionaryMatch(d=diseases, longest_match_only=True)
matches = []
for match in matcher.apply(cs.apply(sents[0])):
    matches.append(match)
    print match

<Ngram("Tricuspid valve regurgitation", id=6794356-0:0-28, chars=[0,28], words=[0,2])
<Ngram("toxicity", id=6794356-0:52-59, chars=[52,59], words=[6,6])


In [61]:
matches[1][:5]

<Ngram("toxic", id=6794356-0:52-56, chars=[52,56], words=[6,6])

In [45]:
matcher = DictionaryMatch(d=diseases, longest_match_only=False)
for match in matcher.apply(cs.apply(sents[0])):
    print match

<Ngram("Tricuspid valve regurgitation", id=6794356-0:0-28, chars=[0,28], words=[0,2])
<Ngram("regurgitation", id=6794356-0:16-28, chars=[16,28], words=[2,2])
<Ngram("toxicity", id=6794356-0:52-59, chars=[52,59], words=[6,6])


### Writing a `CDR` candidate extractor

We'll start by writing a _disease mention extractor_, then a _chemical mention extractor_, and finally will take all _C-D_ pairs co-occuring in a sentence as our C-D relation candidates.

#### Loading a gold candidate set
First, we'll load in the gold annotations from the CDR development set, so that we can test our _candidate recall_, in other words how good our candidate extractor's coverage is.

In [ ]:
doc = docs[0]
doc

In [ ]:
from utils import collect_pubtator_annotations
annos = collect_pubtator_annotations(doc)

#### Disease mentions
We'll build our disease mention extractor using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`) and `DDLite`'s `CandidateExtractor` operators.

In [6]:
from load_dictionaries import load_disease_dictionary, \
                              load_acronym_dictionary

In [7]:
diseases = load_disease_dictionary()

In [8]:
len(diseases)

507899

### Comparing against gold candidate set

#### TODO: DictionaryMatch accepts either list or dict; in latter case, assumes vals are the IDs!

#### ALSO: Add estimate_size method to CandidateExtraction operators

### Saving to disk

# TODO: Write tests!